In [ ]:

from datasets import load_from_disk, concatenate_datasets, DatasetDict, Dataset
import IPython.display as ipd
import speech_utils as su
import random
import numpy as np
from transformers import Wav2Vec2CTCTokenizer
from transformers import SeamlessM4TFeatureExtractor
from transformers import Wav2Vec2BertProcessor
from transformers import Wav2Vec2BertForCTC
from transformers import TrainingArguments
from transformers import Trainer

import numpy as np
import pandas as pd
import random
import torch
from datasets import load_metric, Audio

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

datasets_dir = [
    '/mnt/sea/speech/processed_datasets/CMU_Synth_ASR',
      '/mnt/sea/speech/processed_datasets/Common_Voice_16_1_pa-IN_ASR', 
      '/mnt/sea/speech/processed_datasets/fleurs_pa_ASR',
      '/mnt/sea/speech/processed_datasets/Google_Synth_ASR',
       '/mnt/sea/speech/processed_datasets/IndicSuperb_pa_ASR',
       '/mnt/sea/speech/processed_datasets/Indicvoice_pa_ASR',
       '/mnt/sea/speech/processed_datasets/PunjabiSpeech_A_labeled_Speech_Corpus_ASR',
        '/mnt/sea/speech/processed_datasets/shrutilipi_pa_ASR'
]
datasets = [load_from_disk(f'{d}') for d in datasets_dir]

all_data_splits = []
train_data_splits = []
train_valid_data_splits = []
test_data_splits = []
valid_data_splits = []


train_splits_log = []
test_splits_log = []
for d, ds_dir in zip(datasets, datasets_dir):
    for split in d:
        all_data_splits.append(d[split])
        if split == 'train':
            train_data_splits.append(d[split])
        if split == 'train' or 'valid' in split:
            train_splits_log.append(f'{ds_dir.split('/')[-1]} - {split}')
            train_valid_data_splits.append(d[split])
        if split == 'test':
            test_splits_log.append(f'{ds_dir.split('/')[-1]} - {split}')
            test_data_splits.append(d[split])
        if 'valid' in split:
            valid_data_splits.append(d[split])

print(len(all_data_splits))
print(len(train_data_splits))
print(len(train_valid_data_splits))
print(len(test_data_splits))
print(len(valid_data_splits))

print("Train Data Splits")
print(train_splits_log)
print("Test Data Splits")
print(test_splits_log)


ds_all = concatenate_datasets(all_data_splits)
ds_train = concatenate_datasets(train_data_splits)
ds_train_valid = concatenate_datasets(train_valid_data_splits)
ds_test = concatenate_datasets(test_data_splits)
ds_valid = concatenate_datasets(valid_data_splits)

print(ds_all)
print(ds_train)
print(ds_train_valid)
print(ds_test)
print(ds_valid)

ds = DatasetDict({
    'train': ds_train_valid,
    'test': ds_test,
})

In [ ]:
# INDICTTS TEST
dir = ['/mnt/sea/speech/', '/mnt/sea/speech/benchmarks/vistaar/benchmarks/']
d = ['indictts_ds', 'fleurs']

dir = dir[1]
d = d[1]

su.print_red(f'Processing {d}...')
manifest = f'{d}/punjabi/manifest.json' # path in manifest is {d}/punjabi/wavs/
df = pd.read_json(f'{dir}{manifest}', lines=True)
df['audio_filepath'] = df['audio_filepath'].apply(lambda x: f'{dir}{x}')
df = df.rename(columns={'audio_filepath': 'audio'})
ds = Dataset.from_pandas(df.reset_index(drop=True))
ds = ds.cast_column('audio', Audio(sampling_rate = 16000))
if 'test' not in ds.column_names:
    ds = ds.train_test_split(test_size=0.01, seed=42)
    print(ds)

In [ ]:
# ds = load_from_disk('/mnt/sea/speech/processed_datasets/IndicSuperb_pa_ASR')

In [ ]:
su.get_summary(ds)

# if train and valid splits are there - change name of valid split to test
if 'noisy_test' in ds.column_names:
    train = concatenate_datasets([ds['train'], ds['valid']])
    test = concatenate_datasets([ds['test'], ds['test_known'], ds['noisy_test'], ds['noisy_test_known']])
    ds = DatasetDict({'train': train, 'test': test})

if 'valid' in ds.column_names and 'test' not in ds.column_names:
    ds['test'] = ds['valid']
    del ds['valid']

# split ['train'] to ['train', 'test']
if 'test' not in ds.column_names:
    ds = ds['train'].train_test_split(test_size=0.04, seed=42)
    print(ds)

if 'valid' in ds.column_names:
    ds = su.merge_train_valid_splits(ds)

ds = su.add_silence(ds)
ds['train'] = su.remove_audio_samples(ds['train'])
ds['test'] = su.remove_audio_samples(ds['test'])
ds = su.normalize_text_ds(ds)
ds['train'] = su.remove_text_samples(ds['train'], column_name='normalized_text')
ds['test'] = su.remove_text_samples(ds['test'], column_name='normalized_text')
su.get_summary(ds, text_column='normalized_text')

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")
processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

def prepare_dataset(batch):
    # batch is single row
    audio = batch["audio"]

    clean_audio_arr = audio["array"]
    noised_audio = clean_audio_arr

    batch["input_features"] = processor(noised_audio, sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    batch["labels"] = processor(text=batch["normalized_text"]).input_ids
    return batch


In [13]:
individual_wer = False # if True, calculate WER for each audio file

eval_split = 'train'
ckpt = 'wav2vec2-bert-pa_4/checkpoint-12300'

In [4]:
from jiwer import compute_measures
import datasets as datasets_lib
class CustomWER(datasets_lib.Metric):
    def _info(self):
        return datasets_lib.MetricInfo(
            description='_DESCRIPTION',
            citation='_CITATION',
            inputs_description='_KWARGS_DESCRIPTION',
            features=datasets_lib.Features(
                {
                    "predictions": datasets_lib.Value("string", id="sequence"),
                    "references": datasets_lib.Value("string", id="sequence"),
                }
            ),
            codebase_urls=["https://github.com/jitsi/jiwer/"],
            reference_urls=[
                "https://en.wikipedia.org/wiki/Word_error_rate",
            ],
        )

    def _compute(self, predictions=None, references=None, concatenate_texts=False):
        wers = []
        for prediction, reference in zip(predictions, references):
            incorrect = 0
            total = 0
            try:
                measures = compute_measures(reference, prediction)
                incorrect += measures["substitutions"] + measures["deletions"] + measures["insertions"]
                total += measures["substitutions"] + measures["deletions"] + measures["hits"]
                wers.append(incorrect / total)
            except Exception as e:
                wers.append(1000.0)
                print(f"Error in WER calculation for {prediction} and {reference}")

        
        # save wers to file - For testing purposes
        with open(f"{eval_split}_wers.txt", "a") as f:
            f.write(f"{wers}\n")
        return wers


In [5]:

# for manual evaluation/testing of specific split
if eval_split == 'test':
    dsp_train = ds['test'].map(prepare_dataset, remove_columns=ds['test'].column_names, num_proc=1, batch_size=64, writer_batch_size=64, )
    dsp_test = dsp_train
elif eval_split == 'train':
    dsp_train = ds['train'].map(prepare_dataset, remove_columns=ds['train'].column_names, num_proc=1, batch_size=64, writer_batch_size=64, )
    dsp_test = dsp_train
else:
    dsp_train = ds['train'].map(prepare_dataset, remove_columns=ds['train'].column_names, num_proc=1, batch_size=64, writer_batch_size=64, )
    dsp_test = ds['test'].map(prepare_dataset, remove_columns=ds['test'].column_names, num_proc=1, batch_size=64, writer_batch_size=64, )


dsp = DatasetDict({
    'train': dsp_train,
    'test': dsp_test,
})

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2BertProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

if individual_wer:
    wer_metric = CustomWER()
else:
    wer_metric = load_metric("wer")


model = Wav2Vec2BertForCTC.from_pretrained(
    ckpt,
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

batch_size = 16
accumulation_steps = 1

effective_batch_size = batch_size * accumulation_steps


training_args = TrainingArguments(
  output_dir='./wav2vec2-bert-pa_eval',
  per_device_train_batch_size=batch_size,
  gradient_accumulation_steps=accumulation_steps,
  per_device_eval_batch_size=6,
  eval_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=10,
  adam_beta1=0.9,
  adam_beta2=0.999,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=5,
  learning_rate=5e-5,
  lr_scheduler_type="cosine",
  load_best_model_at_end=True,
  metric_for_best_model="wer",
  greater_is_better=False,
  ignore_data_skip=True,
  save_total_limit=4,
  push_to_hub=False,
  report_to= "none",
)
from functools import partial

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=partial(su.compute_wer_metrics, processor=processor),
    train_dataset=dsp['train'],
    eval_dataset=dsp['test'],
    tokenizer=processor.feature_extractor,
)


Map:   0%|          | 0/465 [00:00<?, ? examples/s]

/home/kd/anaconda3/envs/hf2/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


cuda:0


In [21]:
test_range = 465
dspe = dsp[eval_split]
dse = ds[eval_split]

if test_range != 0:
    if test_range > 0:
        dspe = dspe.select(range(0, test_range))
        dse = dse.select(range(0, test_range))
    else:
        dspe = dspe.select(range(len(dspe) + test_range, len(dspe)))
        dse = dse.select(range(len(dse) + test_range, len(dse)))


if not individual_wer:
    wers = trainer.evaluate(eval_dataset=dspe,)['eval_wer']
    print(len(wers))
    len(dse), len(wers)

    werss = list(enumerate(wers))
    # sort werss by wer
    werss = sorted(werss, key=lambda x: x[1], reverse=True)


### Compare two different KenLM models

In [ ]:
from m4t_processor_with_lm import M4TProcessorWithLM
processor_asr_lm = M4TProcessorWithLM.from_pretrained('/home/kd/Desktop/proj/apr/speech_pa/wav2vec2-bert-pa-lm_processor')
processor_stories_lm = M4TProcessorWithLM.from_pretrained('/home/kd/Desktop/proj/apr/speech_pa/wav2vec2-bert-pa-lm_processor_stories_wiki')

In [ ]:
losses = {}
wers = {}
wers_asr_lm = {}
wers_stories_lm = {}
pred = {}
pred_asr_lm = {}
pred_stories_lm = {}
gt = {}
from tqdm import tqdm
for i in range(len(dspe)):
    print(f'Processing {i}...')
    input_features = torch.tensor(dspe[i]["input_features"]).to("cuda").unsqueeze(0)
    labels = torch.tensor(dspe[i]["labels"]).to("cuda").unsqueeze(0) 

    with torch.no_grad():
        logits = model(input_features, labels=labels)
        # loss as double
        loss = logits.loss.item()
        losses[i] = loss

        pred_ids = torch.argmax(logits.logits, dim=-1)[0]
        pred[i] = processor.decode(pred_ids)

        logits = logits.logits.cpu().detach().numpy()
        pred_asr_lm[i] = processor_asr_lm.batch_decode(logits).text[0]
        pred_stories_lm[i] = processor_stories_lm.batch_decode(logits).text[0]

        wers[i] = wer_metric.compute(predictions=[pred[i]],references= [dse[i]["normalized_text"]])
        wers_asr_lm[i] = wer_metric.compute(predictions=[pred_asr_lm[i]],references= [dse[i]["normalized_text"]])
        wers_stories_lm[i] = wer_metric.compute(predictions=[pred_stories_lm[i]],references= [dse[i]["normalized_text"]])

        # convert wer to percentage with 2 decimal points
        wers[i] = round(wers[i] * 100, 2)
        wers_asr_lm[i] = round(wers_asr_lm[i] * 100, 2)
        wers_stories_lm[i] = round(wers_stories_lm[i] * 100, 2)

        gt[i] = dse[i]["normalized_text"]

        # update with su.pbprint()
        pred[i] = su.pbprint(pred[i])
        pred_asr_lm[i] = su.pbprint(pred_asr_lm[i])
        pred_stories_lm[i] = su.pbprint(pred_stories_lm[i])
        gt[i] = su.pbprint(gt[i])



In [23]:
# create a dataframe
df = pd.DataFrame({
    'ground_truth': gt,
    'prediction': pred,
    'pred_asr_lm': pred_asr_lm,
    'pred_stories_lm': pred_stories_lm,
    'wer_pred': wers,
    'wer_asr_lm': wers_asr_lm,
    'wer_stories_lm': wers_stories_lm,
    'loss': losses,
})
df.head()

,ground_truth,prediction,pred_asr_lm,pred_stories_lm,wer_pred,wer_asr_lm,wer_stories_lm,loss
0,ਬਲੌਗਿੰਗ ਇੱਕ ਅਜਿਹਾ ਸਾਧਨ ਹੈ ਜੋ ਸਹਿਯੋਗ ਦੀ ਪ੍ਰੇਰਣਾ...,ਬਲਾਗਿੰਗ ਇੱਕ ਅਜਿਹਾ ਸਾਧਨ ਹੈ ਜੋ ਸਹਿਯੋਗ ਦੀ ਪ੍ਰੇਰਨਾ...,ਬਲੌਗਿੰਗ ਇੱਕ ਅਜਿਹਾ ਸਾਧਨ ਹੈ ਜੋ ਸਹਿਯੋਗ ਦੀ ਪ੍ਰੇਰਣਾ...,ਬਲੌਗਿੰਗ ਇੱਕ ਅਜਿਹਾ ਸਾਧਨ ਹੈ ਜੋ ਸਹਿਯੋਗ ਦੀ ਪ੍ਰੇਰਣਾ...,15.38,3.85,7.69,0.103631
1,ਯੂਰਪ ਇੱਕ ਅਜਿਹਾ ਮਹਾਂਦੀਪ ਹੈ ਜੋ ਉਂਝ ਤਾਂ ਆਕਾਰ ਵਿੱਚ...,ਯੂਰਪ ਇੱਕ ਅਜਿਹਾ ਮਹਾਂਦੇਵ ਹੈ ਜੋ ਉੰਜ ਦਾ ਘਾਰ ਵਿੱਚ ਛ...,ਯੂਰਪ ਇੱਕ ਅਜਿਹਾ ਮਹਾਂਦੀਪ ਹੈ ਜੋ ਉਂਝ ਦਾ ਘਰ ਵਿੱਚ ਛੋ...,ਯੂਰਪ ਇੱਕ ਅਜਿਹਾ ਮਹਾਂਦੀਪ ਹੈ ਜੋ ਉਂਜ ਦਾ ਘਰ ਵਿੱਚ ਛੋ...,12.20,4.88,7.32,0.211934
2,ਸੱਭਿਅਤਾ ਸ਼ਬਦ ਲੈਟਿਨ ਸੱਭਿਅਤਾ ਤੋਂ ਆਇਆ ਹੈ ਜਿਸਦਾ ਅਰਥ...,ਸੱਭਿਅਤਾ ਸ਼ਬਦ ਲੈਟਿਨ ਸਭਿਅਤਾ ਤੋਂ ਆਇਆ ਹੈ ਜਿਸਦਾ ਅਰਥ ...,ਸੱਭਿਅਤਾ ਸ਼ਬਦ ਲੈਟਿਨ ਸੱਭਿਅਤਾ ਤੋਂ ਆਇਆ ਹੈ ਜਿਸਦਾ ਅਰਥ...,ਸੱਭਿਅਤਾ ਸ਼ਬਦ ਲੈਟਿਨ ਸੱਭਿਅਤਾ ਤੋਂ ਆਇਆ ਹੈ ਜਿਸਦਾ ਅਰਥ...,7.14,0.00,0.00,0.025127
3,ਅਸਟ੍ਰੇਲੀਆ ਦੇ ਮਿਚਲ ਗੌਰਲੀ ਨੇ ਪੁਰਸ਼ਾਂ ਦੇ ਸਟੈਂਡਿੰਗ ...,ਆਸਟ੍ਰੇਲੀਆ ਦੇ ਮਿਚਲ ਗੌਰਲੀ ਨੇ ਪੁਰਸ਼ਾਂ ਦੇ ਸਟੈਂਡਿੰਗ ...,ਆਸਟ੍ਰੇਲੀਆ ਦੇ ਮਿਚਲ ਗੌਰਲੀ ਨੇ ਪੁਰਸ਼ਾਂ ਦੇ ਸਟੈਂਡਿੰਗ ...,ਆਸਟ੍ਰੇਲੀਆ ਦੇ ਮਿਚਲ ਗੋਰਲੀ ਨੇ ਪੁਰਸ਼ਾਂ ਦੇ ਸਟੈਂਡਿੰਗ ...,23.33,3.33,30.00,0.233114
4,ਜਦੋਂ ਜੰਗਲੀ ਜਾਨਵਰਾਂ ਦੀ ਗੱਲ ਆਉਂਦੀ ਹੈ ਤਾਂ ਮੇਡਾਗਾਸ...,ਜਦੋਂ ਜੰਗਲੀ ਜਾਨਵਰਾਂ ਦੀ ਗੱਲ ਆਉਂਦੀ ਹੈ ਤਾਂ ਮੇਡਾ ਗਾ...,ਜਦੋਂ ਜੰਗਲੀ ਜਾਨਵਰਾਂ ਦੀ ਗੱਲ ਆਉਂਦੀ ਹੈ ਤਾਂ ਮੇਡਾਗਾਸ...,ਜਦੋਂ ਜੰਗਲੀ ਜਾਨਵਰਾਂ ਦੀ ਗੱਲ ਆਉਂਦੀ ਹੈ ਤਾਂ ਮੈਡਾਗਾਸ...,9.09,0.00,4.55,0.087635


In [24]:
df.shape

(465, 8)

In [ ]:
# sort by wer_pred
dfx = df.sort_values(by='wer_pred', ascending=False)
for i, row in dfx.iterrows():
    print(f'GT:              {row["ground_truth"]}')
    print(f'PRED:            {row["prediction"]}')
    print(f'PRED ASR LM:     {row["pred_asr_lm"]}')
    print(f'PRED STORIES LM: {row["pred_stories_lm"]}')
    print(f'Loss: {row["loss"]} WER: {row["wer_pred"]} WER ASR LM: {row["wer_asr_lm"]} WER STORIES LM: {row["wer_stories_lm"]} ')
    ipd.display(ipd.Audio(data=dse[i]["audio"]["array"], autoplay=False, rate=16000))
    print('\n\n')


In [28]:
# save as csv
# df.to_csv(f'fluers_benchmark_asr_various_decoding_results.csv', index=False)

### Long Form Speech Transcription

In [72]:
from transformers import pipeline

pr = processor_asr_lm

pipe = pipeline('automatic-speech-recognition', model=model, tokenizer=pr.tokenizer, feature_extractor=pr.feature_extractor, decoder=pr.decoder, return_timestamps='word', device='cuda:0')

output = pipe("dil_full_mono.wav", chunk_length_s=20, stride_length_s=(6, 6))
su.pbprint(output['text'])
# print(output['chunks'])

def convert_to_hms(seconds: float) -> str:
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = round((seconds % 1) * 1000)
    output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
    return output

ਦਲਜੀਤ ਕੰਨ ਖਲ ਕੇ ਮੇਰੀ ਗੱਲ ਸੁਨਲਾ ਦੋ ਮਿੰਟ ਲਿਉ ਕੇ ਤੂੰ ਟੂਰ ਤੇ ਗਿਆ ਜਾਂ ਇਹਨਾਂ ਭੰਗੜਾਂ ਪਾਕੇ ਬੰਦ ਨੂੰ ਭੱਖ ਦਾਂ ਲੱਗਦੀ ਆ ਰੋਟੀ ਖਾਣ ਨੂੰ ਜੀ ਕਰ ਦਾਈ ਆ ਵਰਾਈਸ ਸਾੳ ਮੈਨੂੰ ਹਾਇਰ ਕਰ ਦੋ ਟੀਮ ਤੇ ਯੂਨੀਅਰ ਰੋਟੀ ਮੇਕਰ ਆਵੀਅਸ ਲੀ ਮੈਂ ਦੋ ਮਿੰਟ ਤਾਂ ਬਣਾ ਦੂਗੀ ਦਗਰਦਗਰਦੈ ਖਿਲਾਦੂਗੀ ਆਪ ਬੁਰਕੀਆਂ ਕਰ ਕਰਕੇ ਖਿਲਾ ਦੂਗੀ ਤੇਨੂੰ ਖੇਲ ਲਕਿਂ ਮੈਂ ਤਾਂ ਵੇਲੀ ਆਂ ਮੈਨੂੰ ਤਾਂ ਕੋਈ ਕੰਮ ਹਿਣੀਗਾ ਮੈਂ ਤਾ ਾੱਜੇ ਲਬੀ ਪਈ ਆ ਇਕ ਵੱਜ ਗਿਆ ਸੋ ਲਈ ਆਮ ਜਿਸਿਗ ਇਸ ਇਸ ਗੁਰ ਬਿਸਨਿਸ ਪ੍ਰਾਪ ਸੀ ਆਲਸੋ ਆਿਮ ਵੈਰੀ ਪ੍ਰਿਟੀ ਆ ਦੇਖ ਲੋ ਸਾਹੁਣੀ ਤਾਂ ਮੈਂ ਬਹੁਤ ੀ ਆ ਬਸ ਐਸ ਵੇਲੇ ਸੂਤੀ ਉਠੀ ਊੰ ਆ ਸੋ ਪਲੀਜ਼ ਡੋਂਟ ਮਾਈਂਡ ਆਹ ਦੇਖ ਲੋ ਉਸ ਸ਼ਿਟ ਕਿੱਥੇ ਮਰ ਗਈ ਓਏ


### Generate Transcript as .srt file

In [68]:
def merge_chunks_into_sentences(chunks):
    sentences = []
    current_sentence = {"text": "", "start_time": None, "end_time": None}
    for chunk in chunks:
        if current_sentence["start_time"] is None:
            current_sentence["start_time"] = chunk["timestamp"][0]

        current_sentence["text"] += " " + chunk["text"].strip()

        # Update end time for the current sentence
        current_sentence["end_time"] = chunk["timestamp"][1]

        # Check if the next chunk will break the sentence
        if chunks.index(chunk) < len(chunks) - 1:
            next_chunk = chunks[chunks.index(chunk) + 1]
            if next_chunk["timestamp"][0] - current_sentence["start_time"] > 3.0:
                # If the duration between the next chunk and the start of the sentence is greater than 1 second
                # then consider it as a new sentence
                sentences.append(current_sentence)
                current_sentence = {"text": "", "start_time": None, "end_time": None}
    # Add the last sentence
    sentences.append(current_sentence)
    return sentences

def convert_to_srt(sentences):
    srt_content = ""
    for i, sentence in enumerate(sentences, start=1):
        start = convert_to_hms(sentence["start_time"])
        end = convert_to_hms(sentence["end_time"])
        text = sentence["text"].strip()
        text = su.pbprint(text)
        srt_content += f"{i}\n{start} --> {end}\n{text}\n\n"
    return srt_content

# Assuming 'output' is the dictionary containing chunks
chunks = output['chunks']
sentences = merge_chunks_into_sentences(chunks)

with open("file_asr_30sec.srt", "w", encoding="utf-8") as f:
    f.write(convert_to_srt(sentences))


ਪਹਿਲਾਂ ਵੀ ਲੋਕ ਗੱਲਾਂ ਕਰਦੇ ਸਨ ਵੀ ਤੂੰ ਕੁਛ ਕਰ ਨੀ ਰਿਹਾ
ਤੇ ਜਦੋਂ ਪਗੜੀ ਸੈਂਟਰ ਖੋਲ ਤਾਂ ਉਦੋਂ ਵੀ ਲੋਕੀ ਗੱਲਾਂ ਕਰਨ ਲੈ ਤੂੰ ਚੰਗਾ
ਭਲਾ ਸਰਦਾਰਾਂ ਦਾ ਮੁੰਡਾ ਚੰਗਾ ਭਲਾ ਜੱਟਾਂ ਦਾ ਮੁੰਡਾ ਜ਼ਮੀਨ ਜਾਦਾ
ਤੇਰੇ ਕੋਲ ਤੇ ਤੂੰ ਸੌ ਸੌ ਰੁਪਏ ਬੱਦਲ ਪੱਗ ਬੰਨੇਗਾ ਮੈਂ ਕਹਿੰਦੀ
ਅੱਜ ਦੋ ਸੌ ਕੱਲ੍ਹ ਨੂੰ ਦੋ ਹਜ਼ਾਰ ਹੋਏਗਾ ਤੇ ਪਰ
ਸੋਨੂੰ ਚਾਲੀ ਹਜ਼ਾਰ ਵੀ ਹੋਏਗਾ ਪਹਿਲਾਂ ਤੇ ਮੈਨੂੰ ਇਕ ਮਿੰਟ
ਯਕੀਨੀ ਨੀ ਹੋਇਆ ਵੀ ਦਲਜੀਤ ਭਾਜੀ ਵਾਸਤੇ ਮੈਨੂੰ ਫੋਨ ਆਇਆ ਪੱਗ ਬੰਨਣ
ਦੀ ਜਦੋਂ ਦਸ ਬਾਏ ਬਾਰਾਂ ਦੇ ਕਮਰੇ
ਦੇ ਵਿੱਚ ਵੀਹ ਵੀਹ ਪੱਗਾਂ ਮੈਂ ਸੌ ਸੌ ਦੇ ਹਿਸਾਬ ਨਾਲ
ਬੰਨਾਉਂਦਾ ਸਾਂ ਤੇ ਅੱਜ
ਤੁਸੀਂ ਆ ਗਾਣਾ ਤੇ ਸੁਣਿਆ ਈ ਹੋਣਾ ਆ ਕੇ ਪੱਗਾਂ
ਪੋਛਵੀਆਂ ਵਾਲੇ ਰਾਹੀਂ ਬਚ ਕੇ ਨੀ ਰੰਗਲੇ ਦੁਪੱਟੇ ਵਾਲੀਏ
ਤੁਸੀਂ ਆਹ ਵੀ ਗਾਣਾ ਸੁਣਿਆ ਹੋਣਾ ਮੂਹਰੇ ਬੰਨ੍ਹ ਕੇ ਗਲਾਬੀ ਪੱਗ
ਖੜ੍ਹਦਾ ਨੀ ਪੁੱਤ ਜੱਟਾਂ ਦਾ ਏ ਗੱਭਰੂ ਤੁਸੀਂ ਪੱਗਾਂ ਦੇ ਉੱਤੇ ਗਾਣੇ
ਤਾਂ ਬਹੁਤ ਸੁਣੇ ਹੋਣਗੇ ਪਰ ਕਦੀ ਇਹ ਨਹੀਂ ਸੁਣਿਆ ਹੋਣਾ
ਵੀ ਪੱਗ ਨੇ ਕਿਸੇ ਦੀ ਜ਼ਿੰਦਗੀ ਬਦਲ ਦਿੱਤੀ ਸਤਸ੍ਰੀਕਾਲ ਜੀ
ਸਾਰਿਆਂ ਨੂੰ ਮੈਂ ਗੁਰਪ੍ਰਤਾਪ ਸਿੰਘ ਕੰਗ ਪਿੰਡ ਰਣਜੀਤ
ਨਗਰ ਗੌਂਦਰ ਜ਼ਿਲ੍ਹਾ ਕਰਨਾਲ ਹਰਿਆਣਾ
ਮੈਂ ਤਹਾਨੂੰ ਦੱਸਣਾ ਆ ਕਿਵੇਂ ਪੱਗ ਨੇ ਮੇਰੀ ਜ਼ਿੰਦਗੀ ਬਦਲ
ਦਿੱਤੀ ਮੈਨੂੰ ਸ਼ੁਰੂ ਤੋਂ ਹੀ ਪੱਗ ਦਾ ਬੜਾ ਸ਼ੌਂਕ ਸੀ
ਬੜਾ ਚਾਅ ਸੀ ਅਕਸਰ ਮੈਂ ਆਪਣੇ ਫਾਦਰ ਸਾਹਿਬ ਨੂੰ
ਵੀ ਪੱਗ ਬੰਨ ਦਿਆਂ ਵੇਂਦਾ ਹੁੰਦਾ ਸਾਂ ਤੇ ਫਿਰ ਮੈਂ

### Inspect examples after soring 

In [ ]:
def show_example(index):
    print(f'Index: {index}, WER: {wers[index]}')
    # print(f'Normalized Text: {dse[index]["normalized_text"]}')
    input_features = torch.tensor(dspe[index]["input_features"]).to("cuda").unsqueeze(0)
    with torch.no_grad():
        logits = model(input_features).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    su.pbprint(f'Prediction: {processor.decode(pred_ids)}')
    su.pbprint(f'Ground Truth: {processor.decode(dspe[index]["labels"]).lower()}')
    ipd.display(ipd.Audio(data=dse[index]["audio"]["array"], autoplay=False, rate=16000))


count = 0
for i, wer in werss:
    if count < 100 and count > 0:
        # print( wer)
        show_example(i)
    count += 1